In [1]:
!pip install langchain
!pip install langchain-chroma
!pip install bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers
!pip install -q -U git+https://github.com/huggingface/accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.1/289.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
import torch
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [3]:
from huggingface_hub import login
login() #login and get your Token https://huggingface.co/settings/tokens

In [4]:
#bitandbytes helps to load bigger parameter models in quantize version. OG size of the Model required Higer GPU. for llama-2 min it required 20+ GPU
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False)

In [5]:
model_id = "meta-llama/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config = bnb_config,
                                             device_map="cuda", #change here "cpu" if you don't have GPU
                                             )

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
#use of Tokenizer
#encoding the input text for machine understanding language
text = "Hi I'am Nobody"

encoded_text = tokenizer(text)["input_ids"]

print(encoded_text)

[1, 6324, 306, 29915, 314, 15706, 1486]


In [7]:
#decoding back into human understand language
decoded_text = tokenizer.decode(encoded_text)

print("Decoded text: ", decoded_text)

Decoded text:  <s> Hi I'am Nobody


In [8]:
#creating pipeline with necessary parameters
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0.1,
    top_p=0.95,
    repetition_penalty=1.15
)

In [11]:
llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={"temperature": 0})

In [12]:
llm("what is python?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'what is python?\n Unterscheidung between Python 2.x and Python 3.x.\n\nPython is a high-level, interpreted programming language that was created in the late 1980s by Guido van Rossum. It is known for its simplicity, readability, and ease of use, making it a popular choice among developers and beginners alike. Python is often used for web development, scientific computing, data analysis, artificial intelligence, machine learning, and more.\n\nThere are two main versions of Python: Python 2.x and Python 3.x. The most significant difference between these two versions is their syntax and compatibility with older code. Here\'s a brief overview of each version:\n\nPython 2.x (also called "classic Python"):\n\n* Released in 2000\n* Used until around 2020\n* Supported until around 2025\n\nPython 2.x has a different syntax than Python 3.x, which means that code written in Python 2.x will not work as-is on Python 3.x. This includes differences in indentation, string formatting, and other aspect

In [13]:
llm("can you tell me a joke about life?")

'can you tell me a joke about life?\n everybody loves to laugh, and jokes are a great way to bring people together. Here is one that I think you might enjoy:\n\nWhy did the scarecrow win an award? Because he was outstanding in his field!\n\nI hope this made you smile! Do you have any favorite jokes or topics that you would like to talk about?'